## CNN实现手写识别 （18/06/06）

In [2]:
# 导入mnist的数据集
from tensorflow.examples.tutorials.mnist import input_data
# read_data_sets(train_dir,fake_data=False,one_hot=True,dtype=dtypes.float32,
                   #reshape=True, validation_size=5000,seed=None)
# 过程就是下载数据，解压数据，返回datasets，包括train，validation,test三个部分
# one_hot = True 是考虑多类情况。非onehot，标签是0 1 2 3 这样的
# one_hot 就是一个长度为 n 的数组，只有一个元素是1，其他元素是0
# mnist的标签就是这样的，所以one_hot = True
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
import tensorflow as tf

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [4]:
# tf.placeholder(dtype, shape=None, name=None)
# 此函数可以理解为形参，用于定义过程，在执行的时候再赋具体的值
# shape：数据形状。比如[2,3], [None, 10]表示列是10，行不定

x = tf.placeholder(tf.float32, [None, 784])  
y_ = tf.placeholder(tf.float32, [None, 10])  

# TensorFlow程序的流程是先创建一个图，然后在session中启动它
sess = tf.InteractiveSession()  

# 用于初始化 W
def weight_variable(shape):  
    initial = tf.truncated_normal(shape, stddev=0.1)  
    return tf.Variable(initial)  

# 用于初始化 b
def bias_variable(shape):  
    initial = tf.constant(0.1, shape=shape)  
    return tf.Variable(initial)  

# 用于构建卷积层  
# tf.nn.conv2d(input, filter, strides, padding）
# input：[batch, in_height, in_width, in_channels]
# filter: [filter_height, filter_width, in_channels, out_channels]
def conv2d(x, W):  
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME') 


# 用于构建池化层  
# tf.nn.max_pool(value, ksize, strides, padding, name=None)
# value：需要池化的输入，一般池化层接在卷积层后面，所以输入通常是feature map
# ksize: 池化窗口的大小，取一个四维向量，一般是[1, height, width, 1]
       # 因为我们不想在batch和channels上做池化，所以这两个维度设为了1
# strides：和卷积类似，窗口在每一个维度上滑动的步长，一般也是[1, stride,stride, 1]
# padding：和卷积类似，可以取'VALID' 或者'SAME'
def max_pool_2x2(x):  
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],  
                          strides=[1, 2, 2, 1], padding='SAME')  
# 池化窗口是2 X 2的
# 整个网络两个卷积层，一个全连接层，一个dropout层，一个softmax层

# 第一层卷积  
W_conv1 = weight_variable([5, 5, 1, 32])  
b_conv1 = bias_variable([32])  

# tf.reshape(tensor, shape, name=None) 
# 函数的作用是将tensor变换为参数shape的形式
# -1 表示，先按28 x 28来 ，然后有多少层，系统安排
x_image = tf.reshape(x, [-1, 28, 28, 1])  
# 第一个卷积层
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)  
# conv2d 根据前面的解释，所以这里的filter是5X5的，通道是32个
# 第一个池化层
h_pool1 = max_pool_2x2(h_conv1)  



# 第二层卷积  
W_conv2 = weight_variable([5, 5, 32, 64])  
b_conv2 = bias_variable([64])  
# 第二个卷积层
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2) 
# filter 还是5X5，通道是64了

# 第二个池化层
h_pool2 = max_pool_2x2(h_conv2) 

# 全连接层  
W_fc1 = weight_variable([7 * 7 * 64, 1024])  
b_fc1 = bias_variable([1024])  

# reshape成向量
h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])  

# 第一个全连接层
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)  

# dropput层  
keep_prob = tf.placeholder(tf.float32)  
# tf.nn.dropout(x, keep_prob, noise_shape=None, seed=None, name=None)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
# keep_prob 是保留概率


# 输出层  
W_fc2 = weight_variable([1024, 10])  
b_fc2 = bias_variable([10])  


# y_predict = tf.matmul(h_fc1_drop, W_fc2) + b_fc2  
# 或变成softmax层 
y_predict=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)


#训练和评估模型  
# cross_entropy

# tf.nn.softmax_cross_entropy_with_logits(logits, labels, name=None)
# logits 就是神经网路最后一层的输出 
# labels就是实际的标签，用这两者做比较
# 这个函数返回的是一个向量，要求loss就还要对向量求均值，所以用reduce_mean
cross_entropy = tf.reduce_mean(  
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_predict)) 
# 用的AdamOptimizer
# 也可以用GradientDescentOptimizer
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)  
# train_step = tf.train.GradientDescentOptimizer(1e-4).minimize(cross_entropy)  

# 算正确率
correct_prediction = tf.equal(tf.argmax(y_predict, 1), tf.argmax(y_, 1))  

# 运行上面这个函数，会生成一组向量，如：[True, False, True, True]。
# 把它映射成浮点数，然后，计算它们的均值
# cast(x, dtype, name=None) ,将x的数据格式转化成dtype
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))  

#开始训练
sess.run(tf.global_variables_initializer())  
for i in range(1000):  
    # 50一组 把mnist上的东西赋给了batch
    # 之后x = batch[0],y_ = batch[1]
    batch = mnist.train.next_batch(50) 
    
    if i % 100 == 0:  
        train_accuracy = accuracy.eval(feed_dict={  
            x: batch[0], y_: batch[1], keep_prob: 1.0})  
        print("step %d, training accuracy %g" % (i, train_accuracy))  
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})  

print("step %d, training accuracy %g" % (i, train_accuracy))  

step 0, training accuracy 0.12
step 100, training accuracy 0.38
step 200, training accuracy 0.48
step 300, training accuracy 0.78
step 400, training accuracy 0.78
step 500, training accuracy 0.78
step 600, training accuracy 0.82
step 700, training accuracy 0.88
step 800, training accuracy 0.92
step 900, training accuracy 0.86
step 999, training accuracy 0.86
